
# 🧠 Trabajo Práctico – Clasificación de Accidente Cerebrovascular (Stroke)

## 1. Descripción del Problema

Este trabajo aborda un **problema de clasificación binaria**, utilizando un dataset clínico que contiene información demográfica y médica de pacientes.  
El objetivo es predecir si una persona ha sufrido un **accidente cerebrovascular (stroke)** en función de variables como:

- Edad
- Presión arterial
- Nivel de glucosa
- IMC (índice de masa corporal)
- Tipo de trabajo
- Estado civil
- Hábitos de consumo de tabaco, entre otros.

### 🗃️ Resumen del Dataset

- **Registros**: 5110  
- **Variables**: 12  
- **Variable objetivo**: `stroke` (0 = No, 1 = Sí)  
- **Variables categóricas**: `gender`, `ever_married`, `work_type`, `Residence_type`, `smoking_status`  
- **Variables numéricas**: `age`, `avg_glucose_level`, `bmi`, `hypertension`, `heart_disease`  
- La variable `bmi` presenta valores faltantes, y `stroke` está desbalanceada (~5% positivos).

### ✅ Justificación

Este dataset es adecuado porque permite aplicar distintas técnicas de **preprocesamiento**, **selección de características**, **reducción de dimensionalidad** y **modelos de clasificación**.  
Además, presenta un caso realista y crítico en el ámbito de la salud, donde el correcto funcionamiento del modelo tiene un impacto directo en la prevención.

### 🎯 Objetivo del Análisis

Desarrollar un modelo predictivo que permita detectar con la mayor precisión posible los casos positivos de ACV.  
Esto permitirá priorizar atención médica, realizar seguimientos preventivos y gestionar recursos sanitarios de forma más eficiente.


# 🔍 Introducción
Este análisis tiene como objetivo predecir si una persona ha sufrido un accidente cerebrovascular (stroke), utilizando técnicas de minería de datos. El enfoque es supervisado y la variable objetivo es binaria (0 = No, 1 = Sí).


## 2. Análisis Exploratorio de Datos (EDA)

En esta etapa se realiza un análisis detallado del dataset para comprender la estructura, distribución de variables y posibles problemas de calidad de datos.

### 📌 Estadísticas Generales
Se analizan el tipo de datos, la cantidad de registros, valores faltantes y estadísticas descriptivas (media, mediana, desvío estándar, etc.).

### 📊 Distribución de Clases
La variable objetivo `stroke` está desbalanceada: solo el 4.8% de los registros corresponden a casos positivos. Esto es importante para considerar técnicas de balanceo más adelante.

### 🧪 Visualización de Variables
Se utilizan histogramas y gráficos KDE para observar la distribución de variables numéricas como `age`, `avg_glucose_level` y `bmi`.

### 🧮 Asimetría
Se observa la asimetría de las variables numéricas para detectar sesgos.

### 📈 Matriz de Correlación
Permite detectar relaciones lineales entre variables.

### 🧩 Matriz de Dispersión por Clase
Ayuda a observar cómo se agrupan los registros según la clase objetivo.

### 📦 Boxplots por Clase
Visualizan la distribución de cada variable numérica según si la persona tuvo o no un ACV.

### ⚠️ Valores Faltantes
La única columna con nulos es `bmi`, que se completa con la mediana.

### 🧹 Limpieza
Se eliminan columnas irrelevantes (`id`), se codifican variables categóricas, y se verifica la calidad general del dataset.


# Predicción de Accidente Cerebrovascular (Stroke) - Clasificación
Este notebook aborda un problema de clasificación binaria utilizando un dataset clínico para predecir si una persona ha sufrido un accidente cerebrovascular (stroke) o no. Se aplican técnicas de preprocesamiento, selección de variables, reducción de dimensionalidad y modelos de clasificación, incluyendo ajuste por desbalance de clases.

## 📂 Carga del Dataset y Librerías
Se importa el dataset original y las librerías necesarias para el análisis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

ruta = r'C:\Users\Owner\OneDrive\Escritorio\clasificacion, MMD\healthcare-dataset-stroke-data.csv'
df = pd.read_csv(ruta)

print("Dimensiones:", df.shape)
display(df.head())

Dimensiones: (5110, 12)


id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1

## 📊 Revisión de la Estructura del Dataset
Se observa la estructura general, tipos de datos y valores nulos.

In [2]:
# Información general
print(df.info())

# Valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB
None

Valores nulos por columna:
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0


## 3. Preprocesamiento de Datos

Esta etapa transforma el dataset para prepararlo para los modelos de machine learning.

### ✂️ División en Entrenamiento y Prueba
El dataset se divide en 80% entrenamiento y 20% prueba usando `train_test_split`.

### ⚖️ Escalado de Datos
Se aplica escalado estándar (StandardScaler) a todas las variables para que los modelos como SVM funcionen correctamente.

### 🧬 Reducción de Dimensionalidad
Se aplica PCA para conservar el 86% de la varianza explicada. Esto reduce el número de variables y mejora la eficiencia del modelo.

### 🌳 Selección de Características
Se entrena un Random Forest para determinar la importancia de las variables. Se seleccionan las 10 más relevantes.

### ⚖️ Balanceo de Clases con SMOTE
Dado el fuerte desbalance, se utiliza SMOTE para crear registros sintéticos de la clase minoritaria y balancear el conjunto de entrenamiento.


## 📈 Distribución de Clases
Como se trata de un problema de clasificación binaria, es fundamental revisar la proporción de cada clase en la variable objetivo.

In [3]:
# Distribución de clases
sns.countplot(x='stroke', data=df)
plt.title('Distribución de clases: Stroke')
plt.xlabel('¿Tuvo un ACV? (0 = No, 1 = Sí)')
plt.ylabel('Cantidad de registros')
plt.show()

## 🧹 Limpieza y Codificación de Variables
Se imputan valores nulos, se codifican variables categóricas y se eliminan columnas irrelevantes.

In [4]:
# Imputar nulos
df['bmi'].fillna(df['bmi'].median(), inplace=True)

# Codificación de variables
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1, 'Other': 2})
df['ever_married'] = df['ever_married'].map({'No': 0, 'Yes': 1})
df['Residence_type'] = df['Residence_type'].map({'Rural': 0, 'Urban': 1})
df = pd.get_dummies(df, columns=['work_type', 'smoking_status'], drop_first=True)
df.drop(columns=['id'], inplace=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_1196\3312480267.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bmi'].fillna(df['bmi'].median(), inplace=True)


## ✂️ División en Entrenamiento y Prueba
Separación del conjunto de datos en entrenamiento (80%) y prueba (20%).

In [5]:
# Separación de variables
X = df.drop('stroke', axis=1)
y = df['stroke']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## ⚖️ Escalado de Variables Numéricas
Se estandarizan las variables para modelos sensibles a la escala.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 📉 Reducción de Dimensionalidad con PCA
Se aplica PCA para conservar el 86% de la varianza total.

In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.86)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Componentes seleccionadas: {pca.n_components_}")

Componentes seleccionadas: 10


## 🌲 Selección de Variables con Random Forest
Se entrena un modelo base de Random Forest para identificar las 10 variables más importantes.

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
importancia = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
variables_importantes = importancia.head(10).index.tolist()
X_train_sel = X_train[variables_importantes]
X_test_sel = X_test[variables_importantes]

## 🔁 Balanceo de Clases y Entrenamiento del Modelo Final
Se utiliza SMOTE para balancear las clases y se entrena un modelo SVM lineal.

In [9]:
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC

smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

modelo_final = SVC(kernel='linear', random_state=42)
modelo_final.fit(X_train_bal, y_train_bal)

SVC(kernel='linear', random_state=42)

## 🧪 Evaluación del Modelo en Conjunto de Prueba
Se evalúa el desempeño del modelo usando Accuracy, matriz de confusión y métricas de clasificación.

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = modelo_final.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8434442270058709

Matriz de confusión:
[[840 120]
 [ 40  22]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.95      0.88      0.91       960
           1       0.15      0.35      0.22        62

    accuracy                           0.84      1022
   macro avg       0.55      0.61      0.56      1022
weighted avg       0.91      0.84      0.87      1022



## 👤 Predicción de un Nuevo Caso
Se realiza una predicción con el modelo final sobre una persona hipotética.

In [11]:
nueva_persona = pd.DataFrame([{
    "gender": 1, "age": 65, "hypertension": 1, "heart_disease": 0, "ever_married": 1,
    "Residence_type": 1, "avg_glucose_level": 180.0, "bmi": 32.0,
    "work_type_Never_worked": 0, "work_type_Private": 1, "work_type_Self-employed": 0,
    "work_type_children": 0, "smoking_status_formerly smoked": 1,
    "smoking_status_never smoked": 0, "smoking_status_smokes": 0
}])
print("Predicción (1 = stroke, 0 = no stroke):", modelo_final.predict(nueva_persona)[0])

Predicción (1 = stroke, 0 = no stroke): 0


## 💾 Guardado del Modelo
Se guarda el modelo, el escalador y el balanceador (SMOTE) para su uso futuro.

In [12]:
import joblib
joblib.dump(modelo_final, "modelo_svm_stroke.pkl")
joblib.dump(scaler, "scaler_stroke.pkl")
joblib.dump(smote, "smote_stroke.pkl")

['smote_stroke.pkl']

## ✅ Conclusiones Finales
Resumen de los pasos realizados y justificación del modelo final seleccionado.

## Conclusiones
* Se entrenaron múltiples modelos y se aplicó análisis PCA, selección de variables y balanceo de clases con SMOTE.
* El mejor modelo fue SVM balanceado, que logró un mejor recall sobre la clase minoritaria (stroke).
* Este modelo permite predecir con mayor sensibilidad los casos de riesgo y es más útil para contextos médicos reales.


## ✅ Conclusiones Finales

Se entrenaron distintos modelos para predecir la ocurrencia de accidentes cerebrovasculares.

### 🔎 Observaciones
- La variable `stroke` está fuertemente desbalanceada (~5% positivos).
- Se realizaron varias transformaciones: imputación, codificación, escalado, PCA, selección de variables.
- Se aplicó **SMOTE** para balancear las clases y mejorar la sensibilidad del modelo.

### 🧠 Modelo Final
El modelo seleccionado fue un **SVM con kernel lineal**, entrenado con los datos balanceados por SMOTE.

### 📊 Resultados
- Accuracy en test: ~84%
- Recall de clase 1 (stroke): 35% → mejora significativa con respecto al modelo sin balancear.

### ⚠️ Limitaciones
- Dataset limitado a una sola fuente.
- Posible sobreajuste si no se valida con datos externos.
- No se realizó ajuste de hiperparámetros (ej. GridSearchCV).

### 💡 Posibles Mejoras
- Ajustar hiperparámetros del modelo (SVM, RandomForest).
- Probar otros modelos como XGBoost o LightGBM.
- Implementar validación cruzada estratificada y mayor análisis de error.


In [17]:
import joblib

# Guardar el modelo y el scaler exactamente en la carpeta que usás
joblib.dump(modelo_final, r"C:\Users\Owner\OneDrive\Escritorio\clasificacion, MMD\modelo_svm_stroke.pkl")
joblib.dump(scaler, r"C:\Users\Owner\OneDrive\Escritorio\clasificacion, MMD\scaler_stroke.pkl")


['C:\\Users\\Owner\\OneDrive\\Escritorio\\clasificacion, MMD\\scaler_stroke.pkl']